# WIKI-SHEAF

@ Alessio Borgi
@ Stefano Previti

### 0: IMPORTING LIBRARIES

In [22]:
!pip install requests wikipedia-api wptools networkx matplotlib

In [2]:
import requests
import json
import pprint
import wikipediaapi  # Library to interact with Wikipedia easily
import wptools      # Library to extract additional Wikipedia data
from urllib.parse import urlparse, unquote

### 1: GATHERING NECESSARY INFORMATION

#### 1.1: WIKIPEDIA INFORMATION EXTRACTION

In [3]:
def get_wikipedia_data(page_url):
    '''
    Function to extract data from an English Wikipedia page.
    It retrieves:
      - Page title
      - Summary
      - Categories
      - Infobox data
      - Internal links
      - External links
      - References and citations
      - Geographical coordinates
      - List of headings (titles and subtitles)
    '''

    # 1) Title extraction.
    parsed_url = urlparse(page_url)
    title = unquote(parsed_url.path.split("/")[-1])

    # 2) Wikipedia API object for English Wikipedia.
    page = wikipediaapi.Wikipedia(
        user_agent="WikiSheaf/1.0",
        language='en'
    ).page(title)

    if not page.exists():
        return None

    # 3) Extracted data Dictionary creation.
    data = {}

    # 4) Title, Summary and Categories Inclusion in the Dictionary.
    data['title'] = page.title
    data['summary'] = page.summary
    data['categories'] = list(page.categories.keys())

    # 5) Infobox, internal & external links, references & citations, geo info
    #    and headings additional information.
    try:

        # Fetch parsed data from Wikipedia.
        wp_page = wptools.page(title, lang="en")
        wp_page.get_parse()
        extra = wp_page.data

        # Extract all the "additional" info.
        data["infobox"] = extra.get("infobox", None)
        data["internal_links"] = extra.get("links", None)
        data["external_links"] = extra.get("externallinks", None)
        data["references"] = extra.get("references", None)
        data["geographical_coordinates"] = extra.get("coordinates", None)
        data["headings"] = extra.get("sections", None)

    except Exception as e:
        print(f"Error fetching extra data from wptools for page '{title}':", e)
        data["infobox"] = None
        data["internal_links"] = None
        data["external_links"] = None
        data["references"] = None
        data["geographical_coordinates"] = None
        data["headings"] = None

    return data


##### 1.1.1: WIKIPEDIA INFORMATION EXTRACTION - EXAMPLE USAGE

In [13]:
print("STARTING TO EXTRACT FROM WIKIPEDIA...\n")
wikipedia_url = "https://en.wikipedia.org/wiki/Pizza"
wikipedia_data = get_wikipedia_data(wikipedia_url)

STARTING TO EXTRACT FROM WIKIPEDIA...



en.wikipedia.org (parse) Pizza
en.wikipedia.org (imageinfo) File:Pizza-3007395.jpg
Pizza (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Pizza-300...
  infobox: <dict(12)> name, image, image_size, caption, country, r...
  iwlinks: <list(6)> https://commons.wikimedia.org/wiki/Category:P...
  pageid: 24768
  parsetree: <str(77266)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Pizza
  wikibase: Q177
  wikidata_url: https://www.wikidata.org/wiki/Q177
  wikitext: <str(61267)> {{Short description|Italian dish with a f...
}


In [14]:
print("PRINTING DATA EXTRACTED FROM WIKIPEDIA...\n")
pprint.pprint(wikipedia_data)

PRINTING DATA EXTRACTED FROM WIKIPEDIA...

{'categories': ['Category:All Wikipedia articles written in American English',
                'Category:All articles with vague or ambiguous time',
                'Category:Articles with short description',
                'Category:CS1 Greek-language sources (el)',
                'Category:CS1 Italian-language sources (it)',
                'Category:CS1 Spanish-language sources (es)',
                'Category:CS1 Swedish-language sources (sv)',
                'Category:CS1 uses Greek-language script (el)',
                'Category:Cheese dishes',
                'Category:Commons category link is on Wikidata',
                'Category:Convenience foods',
                'Category:Flatbread dishes',
                'Category:Food combinations',
                'Category:Italian cuisine',
                'Category:Italian inventions',
                'Category:Mediterranean cuisine',
                'Category:National dishes',
         

#### 1.2: WIKIDATA INFORMATION EXTRACTION

In [19]:
def parse_wikidata_id(wikidata_url):
    '''
    Helper function to extract the Wikidata ID from a given Wikidata URL.
    '''
    return unquote(urlparse(wikidata_url).path.split("/")[-1])

def get_wikidata_data(wikidata_url):
    '''
    Function to extract the complete information from a Wikidata record.
    It retrieves:
          - Label
          - Description
          - Aliases (alternative names)
          - Instance of (P31)
          - Subclass of (P279)
          - Inception (P571)
          - Image (P18)
          - Country (P17)
          - Official website (P856)
          - Creator/Author (P170)
          - Location (P276)
          - Genre/Style (P136)
      It then extracts the English Wikipedia URL from the sitelinks and calls
      get_wikipedia_data() to fetch Wikipedia page info.
    '''
    # 1) Get the Wikidata ID.
    wikidata_id = parse_wikidata_id(wikidata_url)
    URL = f"https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json"

    # 2) Retrieve all the Wikidata.
    try:
        # 2.1) Fetch data from Wikidata API.
        response = requests.get(URL)
        data = response.json()
        entity = data.get("entities", {}).get(wikidata_id, {})
        wikidata_info = {}
        wikidata_info["id"] = wikidata_id

        # 2.2) Retrieve basic labels, descriptions.
        wikidata_info["label"] = entity.get("labels", {}).get("en", {}).get("value", None)
        wikidata_info["description"] = entity.get("descriptions", {}).get("en", {}).get("value", None)

        # 2.3) Retrieve aliases (alternative names) in English.
        if "aliases" in entity and "en" in entity["aliases"]:
            wikidata_info["aliases"] = [alias.get("value") for alias in entity["aliases"]["en"]]
        else:
            wikidata_info["aliases"] = None

        # 2.4) Extract claims (properties (each one may include multiple values)).
        claims = entity.get("claims", {})

        # 2.4.1) "Instance of" (P31).
        if "P31" in claims:
            wikidata_info["instance_of"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P31"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["instance_of"] = None

        # 2.4.2) "Subclass of" (P279).
        if "P279" in claims:
            wikidata_info["subclass_of"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P279"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["subclass_of"] = None

        # 2.4.3) "Inception" (P571).
        if "P571" in claims:
            wikidata_info["inception"] = [
                claim["mainsnak"]["datavalue"]["value"]["time"]
                for claim in claims["P571"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["inception"] = None

        # 2.4.4) "Image" (P18).
        if "P18" in claims:
            wikidata_info["image"] = [
                claim["mainsnak"]["datavalue"]["value"]
                for claim in claims["P18"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["image"] = None

        # 2.4.5) "Country" (P17).
        if "P17" in claims:
            wikidata_info["country"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P17"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["country"] = None

        # 2.4.6) "Official website" (P856).
        if "P856" in claims:
            wikidata_info["official_website"] = [
                claim["mainsnak"]["datavalue"]["value"]
                for claim in claims["P856"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["official_website"] = None

        # 2.4.7) "Creator/Author" (P170). (for cultural items like art or literature).
        if "P170" in claims:
            wikidata_info["creator"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P170"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["creator"] = None

        # 2.4.8) "Location" (P276).
        if "P276" in claims:
            wikidata_info["location"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P276"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["location"] = None

        # 2.4.9) "Genre/Style" (P136).
        if "P136" in claims:
            wikidata_info["genre"] = [
                claim["mainsnak"]["datavalue"]["value"]["id"]
                for claim in claims["P136"]
                if "mainsnak" in claim and "datavalue" in claim["mainsnak"]
            ]
        else:
            wikidata_info["genre"] = None

        # 2.5) Extract English Wikipedia URL and call the get_wikipedia_data() function.
        if "sitelinks" in entity and "enwiki" in entity["sitelinks"]:
            wikipedia_url = entity["sitelinks"]["enwiki"].get("url", None)
        else:
            wikipedia_url = None
        wikidata_info["wikipedia_url"] = wikipedia_url

        if wikipedia_url:
            wiki_data = get_wikipedia_data(wikipedia_url)
        else:
            wiki_data = None
        wikidata_info["wikipedia_data"] = wiki_data

        return wikidata_info

    except Exception as e:
        print(f"Error fetching Wikidata data for {wikidata_url}: {e}")
        return None

#### 1.2.1: WIKIDATA INFORMATION EXTRACTION - EXAMPLE USAGE

In [20]:
wikidata_url = "https://www.wikidata.org/wiki/Q177" # (Pizza)
wikidata_record = get_wikidata_data(wikidata_url)

en.wikipedia.org (parse) Pizza
en.wikipedia.org (imageinfo) File:Pizza-3007395.jpg
Pizza (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Pizza-300...
  infobox: <dict(12)> name, image, image_size, caption, country, r...
  iwlinks: <list(6)> https://commons.wikimedia.org/wiki/Category:P...
  pageid: 24768
  parsetree: <str(77266)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Pizza
  wikibase: Q177
  wikidata_url: https://www.wikidata.org/wiki/Q177
  wikitext: <str(61267)> {{Short description|Italian dish with a f...
}


In [21]:
pprint.pprint(wikidata_record)

{'aliases': ['pie', 'pizza pie'],
 'country': None,
 'creator': None,
 'description': 'Italian universal popular dish with a flat dough-based base '
                'and toppings',
 'genre': None,
 'id': 'Q177',
 'image': ['Eq it-na pizza-margherita sep2005 sml.jpg', 'Pizza-3007395.jpg'],
 'inception': None,
 'instance_of': ['Q19861951'],
 'label': 'pizza',
 'location': None,
 'official_website': None,
 'subclass_of': ['Q13030962', 'Q1408229', 'Q749316', 'Q2251745', 'Q26996677'],
 'wikipedia_data': {'categories': ['Category:All Wikipedia articles written in '
                                   'American English',
                                   'Category:All articles with vague or '
                                   'ambiguous time',
                                   'Category:Articles with short description',
                                   'Category:CS1 Greek-language sources (el)',
                                   'Category:CS1 Italian-language sources (it)',
             

### 2: BUILDING THE GRAPHS

### 3: HETEROPHILY INDEX

### 4: SHEAF NEURAL NETWORK